# Notebook 4 - Language Models

In this notebook we'll go through the process of building a small n-gram language model that makes some very famous Tweets

This builds very heavily on the work of [Aron Culotta](https://cs.tulane.edu/~aculotta/) at Tulane University.

- How do we score one sentence as more likely than another?
- Will use some probability:
  - $P($"I'm sorry Dave"$) > P($ "Sorry Dave I'm" $)$
- How can we model the probability of a sentence?

<br><br><br>

- Why is it important to be able to score the probability of sentences?

<br><br><br><br>

Example: **Predictive text**

![figs/predict.png](https://github.com/tulane-cmps6730/main/blob/main/lec/language_models/figs/predict.png?raw=1)

![figs/unicorn.jpg](https://github.com/tulane-cmps6730/main/blob/main/lec/language_models/figs/unicorn.jpg?raw=1)

![figs/now.jpg](https://github.com/tulane-cmps6730/main/blob/main/lec/language_models/figs/now.jpg?raw=1)


<br><br><br>

**Many other applications:**


- Machine translation

![tranl](https://github.com/tulane-cmps6730/main/blob/main/lec/language_models/figs/tranl.png?raw=1)

> $P($ "he briefed reporters on the main contents of the statement" $) $ $ > $  
> $P($ "he briefed to reporters the main contents of the statement" $)$

<br><br><br>

- Speech recognition

> $P($ "I am hungry" $) $ $ > P($ "Eye am hungry" $)$

<br><br><br>

- Language generation

![kingcake](https://github.com/tulane-cmps6730/main/blob/main/lec/language_models/figs/kingcake.png?raw=1)

<br><br><br>

**More generally:**

- Due to the prevalence of ambiguity, we need a way to rank probable interpretations of a sentence
- In the future, we'll look at probability of parses
- Today, we'll start with a simpler task
  - The probability of a sentence (word sequence).

<br><br><br>

$p(w_1 \ldots w_m)=$?

E.g., $p($"Sam I am.$")=?$


Given a large sample of sentences $D$, how can we estimate the probability of one particular sentence?

<br><br>
**Analogy**: Given a sample of $n$ people, how do we estimate $p($ brown eyes $)$?

<br><br>
$p($ brown eyes $)= \#($brown eyes$) $ $ / $ $n$  
= "the fraction of all people that have brown eyes"


<br>
So, returning to the above,

$p($Sam I am.$)=$ the fraction of all sentences that equal "Sam I am."

**Why is this a bad idea?**

<br><br><br><br>
Most sentences occur only once.  
$p(w_1 \ldots w_m) \approx \frac{1}{n}$

Any sentence not in $D$ will have probability 0.

Instead, to make estimating these probabilities possible, we need to make assumptions. E.g.,

## Independnece Assumption

Recall from probability: if $A$ and $B$ are independent events, then $P(A,B) = P(A) * P(B)$
- E.g., P("It will rain tomorrow in Chicago" AND " My knee hurts today") =  
P("It will rain tomorrow in Chicago") * P("My knee hurts today")

If we assume that each word in a sentence is an independent event then:

$p(w_1 \ldots w_m) \approx p(w_1) * p(w_2) * \ldots p(w_m)$

(Decompose the joint probability of the words in a sentence into the product of individual word probabilities.)



This is a bad assumption: **why?**
<br><br><br><br>
- "San Francisco", "love you", ...

- we will return to this problem later today

But first: how do we estimate the probability of a single word $p(w_i)$ given a dataset $D$?
<br><br><br><br>

#Estimating Unigram Probabilities


**unigram:** a single word; or, a phrase of length 1.

To estimate unigram probability from some dataset of text, just count the word frequency:

$$ p(w_i) = \frac{C(w_i)}{T} $$

- $C(w_i)=$ number of times word $w_i$ appears in data
- $T=$ number of tokens in data

Recall definition of probability:
- $0 \le p(w_i) \le 1$ $ \forall i$
- $\sum_i p(w_i) = 1$

This is an example of a **<font color="blue">multinomial distribution</font>**:
- A distribution over $n$ discrete events
- In each trial, only one event can occur.
- E.g., roll of an $n$-sided die.
- Imagine a giant die with a word on each face. Each word is weighted by its probability. We can sample words by rolling the die.

<br>

```
<s> I am Sam </s>  
<s> Sam I am </s>  
<s> I do not like green eggs and ham </s>
```

- $p($I$)$ $ = \frac{3}{20} = 0.15$
- $p($Sam$)$ $ = \frac{2}{20} = 0.1$
- ...

## Unigram Implementation

In [ ]:
# common imports
import matplotlib.pyplot as plt
import numpy as np
from numpy import array as npa
import pandas as pd
# Counter: an enhanced dict for counting
# https://docs.python.org/3/library/collections.html
from collections import Counter

d = Counter()
d.update(['a', 'b', 'c'])
d.update(['a', 'a', 'b'])
d

In [ ]:
# Example of estimating unigram probabilities from a set of documents.

docs = ['<s> I am Sam </s>',
        '<s> Sam I am </s>',
        '<s> I do not like green eggs and ham </s>']

def estimate_unigram_probs(docs):
    """
    Compute p(w), the probability of each unigram in this data.
    p(w) = count(w) / total_num_tokens
    """
    counts = Counter()
    for doc in docs:
        tokens = doc.split()   # split sentence into words using spaces
        counts.update(tokens)  # increments counts for all items in tokens
    print('counts=', counts)   # for debugging
    # Normalize so probabilities sum to 1.
    total_tokens = sum(counts.values())
    return {token: value / total_tokens for
                                        token, value in counts.items()}

unigram_probs = estimate_unigram_probs(docs)
# print in descending order.
sorted(unigram_probs.items(), key=lambda x: x[1], reverse=True)

## Probability of a Whole Sentence


Once we've estimated unigram probabilities, we can now compute the probability of a new sentence with $m$ words by the product of its unigram probabilities.

$p(w_1 \ldots w_m) \approx p(w_1) * p(w_2) * \ldots p(w_m)$

(True because of our earlier independence assumption.)

In [ ]:
def sentence_probability(unigrams, sentence):
    proba = 1
    for word in sentence:
        proba *= unigrams[word]
    return proba

import math
def sentence_log_probability(unigrams, sentence):
    proba = 0
    for word in sentence:
        proba += math.log10(unigrams[word])
    return proba

sentence_probability(unigram_probs, ['I', 'Sam', 'am'])

In [ ]:
sentence_probability(unigram_probs, ['Sam', 'I', 'I'])

In [ ]:
# What happens for very very long sentences?

sentence_probability(unigram_probs, ['Sam'] * 1000)

## N-gram Language Models

$p(w_1 \ldots w_m) \approx p(w_1) * p(w_2) * \ldots p(w_m)$


Why is this a bad assumption?

<br><br><br>
Ignores all structure of language

In [ ]:
## e.g., cannot tell that the first sentence is more likely than the second.
sentence_log_probability(unigram_probs, ['Sam', 'I', 'am']) == \
sentence_log_probability(unigram_probs, ['Sam', 'Sam', 'I'])

Really, we'd like a rich syntactic/semantic representation with probabilities assigned.  


We'll get to more complex representations in the future, but for now, we can do better by at least modeling **n-grams**, instead of just unigrams.

## The Markov Assumption

What is the probability of the $i$th word in a sentence, given all the previous words?
$$ p(w_i  | w_1 w_2 \ldots w_{i-1})?$$

We will make an assumption that the $i$th word depends only on the previous $n$ words (**Markov** assumption):

$$ p(w_i | w_1 w_2 \ldots w_{i-1}) \approx p(w_i | w_{i-1} w_{i-2} \ldots w_{i-n}) $$

e.g., 4-grams:

$p($ "flu" $ | $ "He is sick with the" $ ) $ $\approx p($"flu"$ | $ "sick with the" $)$

$$ p(w_i | w_{i-1} w_{i-2} \ldots w_{i-n}) = \frac{C(w_{i-n}, \ldots, w_{i-2} w_{i-1} w_i)}{C(w_{i-n} \ldots w_{i-2} w_{i-1})} $$

<br>
Can estimate from data. E.g., search engines:

= Num hits for ["sick with the flu"](https://www.google.com/search?q=%22i+am+sick+with+the+flu%22) / Num hits for ["sick with the"](https://www.google.com/search?q=%22i+am+sick+with+the%22)  
$ \approx 18,600 / 82,500 \approx 0.23$

<br><br>
See also [Google's NGram corpus](https://books.google.com/ngrams/graph?content=sick+with+the+flu+%2F+sick+with+the&year_start=1800&year_end=2000&corpus=15&smoothing=3&share=&direct_url=t1%3B%2C%28sick%20with%20the%20flu%20/%20sick%20with%20the%29%3B%2Cc0)
- Ngram statistics extracted from millions of books.




## N-Gram Model Implmentation

Let's start with a simpler example:


```
<s> I am Sam </s>  
<s> Sam I am </s>  
<s> I do not like green eggs and ham <s>
```

In [ ]:
def iter_ngrams(doc, n):
    """Return a generator over ngrams of a document.
    Params:
      doc...list of tokens
      n.....size of ngrams"""
    return (doc[i : i+n] for i in range(len(doc)-n+1))

[ngram for ngram in
           iter_ngrams(['<s>', 'hi', 'there', 'how', 'are', 'you', '</s>'], 3)]

In [ ]:
from collections import defaultdict
def estimate_ngram_probs(docs, n, verbose=True):
    """Compute p(w_i|w_i-1, w_i-2, ..., w_i-n), the probability of each ngram in this data.
    = count(w_i-1, w_i-2, ..., w_i-n, w_i) / count(w_i-1, w_i-2, ..., w_i-n)

    Params:
      docs...list of strings
      n......ngram size"""
    # Count
    counts = defaultdict(lambda: Counter())
    for doc in docs:
        for ngram in iter_ngrams(doc.split(), n):
            counts[tuple(ngram[:-1])].update([ngram[-1]])

    if verbose:
        print('counts=\n', '\n'.join(str(i) for i in counts.items()))   # for debugging

    # Normalize probabilities to sum to 1.0
    for ngram, word_counts in counts.items():
        total = sum(word_counts.values())
        counts[ngram] = {word: count / total
                         for word, count in word_counts.items()}
    return counts

sam_ngrams = estimate_ngram_probs(docs, 3)
sam_ngrams

In [ ]:
estimate_ngram_probs(docs, 2)

In [ ]:
# Pr(Sam | "I am")
sam_ngrams[('I', 'am')]['Sam']

## Probability of a Sentence with n-grams

As before, we can compute the probability of a sentence by multiplying probabilities of all ngrams in the sentence.

$$p(w_1 \ldots w_m) = p(w_n|w_{n-1} \ldots w_1) * p(w_{n+1}|w_{n} \ldots w_2) * \ldots * p(w_m|w_{m-1}\ldots w_{m-n})$$

e.g. for $n=2$

$p($ &langle;s&rangle; Sam I am &langle;/s&rangle;$) = p($ Sam $\mid $ &langle;s&rangle; $) * p($I $ \mid $ Sam $) * p($ am $ \mid $ I $) *  p($ &langle;/s&rangle; $ \mid $ am $ ) $

And equivalently in log space:

$$\log p(w_1 \ldots w_m) = \log p(w_n|w_{n-1} \ldots w_1) + \log p(w_{n+1}|w_{n} \ldots w_2) + \ldots + \log p(w_m|w_{m-1} \ldots w_{m-n})$$

In [ ]:
def sentence_log_probability_ngrams(ngrams, sentence, n):
    proba = 0
    # loop through all ngrams
    for ngram in iter_ngrams(sentence, n):
        # e.g., ngram = ['Sam', 'I', 'am']
        #      prefix = ['Sam', 'I']
        #        word = 'am'f
        prefix = ngram[:-1]
        word = ngram[-1]
        # P(wi | ngram)
        this_prob = math.log10(ngrams[tuple(prefix)][word])
        print('Pr(%s|%s)=%g' % (word, prefix, this_prob))
        proba += this_prob
    return proba

sentence_log_probability_ngrams(sam_ngrams, ['<s>', 'Sam', 'I', 'am', '</s>'], 3)

# Let's Make a Donald Trump Bot...

Let's download a sample of Donald Trump's tweets and fit an n-gram language model!

In [ ]:
# download pre-processed data here.
!wget "https://github.com/tulane-cmps6730/main/raw/refs/heads/main/lec/language_models/tweets.txt"

In [ ]:
!head tweets.txt

In [ ]:
trump_tweets = open('tweets.txt').readlines()
print('read %d tweets from Donald J. Trump (@realDonaldTrump)' % len(trump_tweets))
print(trump_tweets[0])

In [ ]:
## Note the second parameter is the total n-gram size, so word + window... if we
## want two words of size then we need to do 3

trump_ngrams = estimate_ngram_probs(trump_tweets, 3, verbose=False)

In [ ]:
# What does Trump think about the media?
sorted(trump_ngrams[('media', 'is')].items(), key=lambda x: -x[1])

In [ ]:
# What does Trump think about the Hillary Clinton?
sorted(trump_ngrams[('clinton', 'is')].items(), key=lambda x: -x[1])

In [ ]:
trump_ngrams[('clinton', 'is')]

# Sampling from ngram Language Model


Given probabilities $p(w_i | w_{i-1} w_{i-2} \ldots w_{i-n})$ $\forall i$, how can we sample sentences?


<br><br><br><br>
1. Sample an ngram $(w_1\ldots w_n)$ where $w_1 = $ &langle;s&rangle;
2. Sample an additional word from $p(w_i | w_{i-1} w_{i-2} \ldots w_{i-n})$
3. Loop until sample &langle;/s&rangle;

In [ ]:
import random
# Sampling uniformly from a list.
random.sample(['a', 'b', 'c'], k=1)

In [ ]:
# Sampling from a multinomial distribution
np.random.choice(['a', 'b', 'c'], size=10, p=[.7, .2, .1])

In [ ]:
# A simple TrumpBot
import random

def generate_sentences(ngrams, k):
    """Sample k sentences from given ngram model.
    Params:
      ngrams....ngram language model; a dict from ngram tuple to a dict
      k.........number of sentences to sample
    """
    # List of all ngrams that start with <s>
    start_ngrams = [ngram for ngram in ngrams if ngram[0] == '<s>']
    for i in range(k):  # sample k sentences.
        # sample uniformly from all start ngrams.
        ngram = random.sample(start_ngrams, 1)[0]
        sentence = []
        sentence.extend(ngram)
        while sentence[-1] != '</s>' and len(sentence) < 15:  # while not at end of sentence.
            # sample the next word
            sampled_word = np.random.choice(list(ngrams[ngram].keys()),      # words
                                            p=list(ngrams[ngram].values()),  # probabilities
                                            size=1)[0]
            sentence.append(sampled_word)
            # update most recent ngram
            ngram = tuple(list(ngram[1:]) + [sampled_word])
        print(' '.join(sentence))

generate_sentences(trump_ngrams, 40)